In [10]:
import pandas as pd
import numpy as np
import json
import requests
import time
import datetime
import os
def get_volatility (limit,symbol,interval):
    data = requests.get('https://fapi.binance.com/fapi/v1/klines', params={"symbol" : symbol,
                                                        "interval" : interval, 
                                                        #"startTime" : startTime,
                                                        #"endTime"   : endTime,
                                                        "limit": limit}).json()
    data  = pd.DataFrame(data,columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', 'no_of_trades', 'tb_base_vol', 'tb_quote_vol', 'ignore' ])
    data.iloc[:,1:]=data.iloc[:,1:].astype(float)
    data['time'] = pd.to_datetime(data['timestamp'], unit='ms')
    volatility = ((data['high']-data['low'])/data['open']).mean()*100 #pct_width
    
    return volatility
def get_top20tickers():
    url = 'https://fapi.binance.com/fapi/v1/ticker/24hr'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    df['quoteVolume'] = df['quoteVolume'].astype(float)
    df =df.sort_values(['quoteVolume'], ascending=False)
    #drop columns
    return df['symbol'].values[:20]
def lev_adjustment(HOC_value):
    BTC_volatility = 0.12 # value of (H-L)/O at the period where you perform the best 
    scale_fac  = HOC_value/BTC_volatility
    max_lev    = 15
    normal_lev = 10
    safe_lev   = 7
    
    SL_best = 0.7  # value of the SL at the period where you perform the best 
    SL = SL_best*scale_fac #calculate stop loss
    return np.round(max_lev/scale_fac,2), np.round(normal_lev/scale_fac,2), np.round(safe_lev/scale_fac,2), np.round(SL,2)

Ticker_    = []
Max_lev    = []
Normal_lev = []
Safe_lev   = []
SL_        = []

for Ticker in get_top20tickers():
    #Ticker = 'BTCUSDT'
    leverage = lev_adjustment(get_volatility(500,Ticker,'5m'))
    Ticker_   .append(Ticker)
    Max_lev   .append(leverage[0])
    Normal_lev.append(leverage[1])
    Safe_lev  .append(leverage[2])
    SL_       .append(leverage[3])
    
    
df = pd.DataFrame(Ticker_,columns=['Ticker'])
df['max']    = Max_lev   
df['normal'] = Normal_lev 
df['safe']   = Safe_lev  
df['SL%']         = SL_       


In [11]:
df

,Ticker,max,normal,safe,SL%
0,BTCUSDT,7.35,4.90,3.43,1.43
1,ETHUSDT,5.07,3.38,2.37,2.07
2,1000LUNCBUSD,0.89,0.59,0.41,11.81
3,BTCBUSD,8.14,5.43,3.80,1.29
4,ETHBUSD,5.45,3.63,2.54,1.93
5,LUNA2BUSD,1.16,0.77,0.54,9.04
6,ETCUSDT,3.30,2.20,1.54,3.18
7,SOLUSDT,4.63,3.09,2.16,2.27
8,ATOMUSDT,2.90,1.94,1.36,3.62
9,EOSUSDT,3.15,2.10,1.47,3.34
